In [39]:
from openai import OpenAI
import os
client = OpenAI(
   api_key=os.environ.get('OPENAI_API_KEY'),
)

In [40]:
my_file = client.files.create(
  file=open("Agreement For Consulting Services.docx", "rb"),
  purpose='assistants'
)

In [41]:
my_assistant = client.beta.assistants.create(
    model="gpt-4o",
    instructions="You are legal contract writting expert.",
    name="Leagl AI",
    tools=[{"type": "file_search"}],
    tool_resources={
    "code_interpreter": {
      "file_ids": [my_file.id]
    }
  }
)

In [42]:
my_thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Create an aggrement for consulting between two parties in the same format as the attached file",
      "attachments": [
        {
          "file_id": my_file.id,
          "tools": [{"type": "file_search"}]
        }
      ]
    }
  ]
)

In [43]:
my_run = client.beta.threads.runs.create(
  thread_id=my_thread.id,
  assistant_id=my_assistant.id,
)

In [44]:
keep_retrieving_run = client.beta.threads.runs.retrieve(
    thread_id=my_thread.id,
    run_id=my_run.id
)




In [45]:
all_messages = client.beta.threads.messages.list(
    thread_id=my_thread.id
)
all_messages

SyncCursorPage[Message](data=[Message(id='msg_V30dL54OqkBxl47Lo5sDM3VT', assistant_id='asst_yXZXvR3DKRt64UQPQ6WAn96V', attachments=[], completed_at=None, content=[], created_at=1718988437, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_a1wcO4myfkNOOG2wlQq1zPAG', status=None, thread_id='thread_4OKouLQkP1vdVnBGsXbU37gL'), Message(id='msg_r2JMRz85upQxBTt83GxgScc4', assistant_id=None, attachments=[Attachment(file_id='file-RJlH9iu44cuw2VgWNBtl1NEF', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create an aggrement for consulting between two parties in the same format as the attached file'), type='text')], created_at=1718988435, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_4OKouLQkP1vdVnBGsXbU37gL')], object='list', first_id=

In [46]:
print(f"Assistant: {all_messages.data[0].content[0].text.value}")

IndexError: list index out of range

In [47]:
import os
import zipfile
import shutil

def unzip_and_cleanup(main_folder):
    for item in os.listdir(main_folder):
        item_path = os.path.join(main_folder, item)
        
        # Check if the item is a zip file
        if item.endswith('.zip'):
            # Unzip the file into a temporary folder
            with zipfile.ZipFile(item_path, 'r') as zip_ref:
                zip_ref.extractall(main_folder)
            
            # Identify and move the contents of any extracted subfolders
            extracted_folders = [os.path.join(main_folder, f) for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]
            for folder in extracted_folders:
                for sub_item in os.listdir(folder):
                    sub_item_path = os.path.join(folder, sub_item)
                    shutil.move(sub_item_path, main_folder)
                
                # Remove the now empty subfolder
                os.rmdir(folder)
            
            # Remove the zip file
            os.remove(item_path)

# Example usage
main_folder = 'contracts/LLC operating agreements'
unzip_and_cleanup(main_folder)
